In [1]:
from time import sleep
import Pyro5.api 
import matplotlib.pyplot as plt
from TimeTagger import *
import numpy as np
import pandas as pd

In [6]:
# Name of the file we want to read. 
# Remember, "Measurement.ttbin" files read all the files in sequence 'Measurement'
file = 'C:/Users/HP/Downloads/TimeTags_PW_Sync_2025-02-20_155351.ttbin' #QD_Sync (PW) 

In [7]:
fr = FileReader(file)
print("All channels available: {}".format(fr.getChannelList()))

print("Has data? ->", fr.hasData())
# Returns: True if more data is available for reading, False if all data has been read from all the files specified in the class constructor.

All channels available: (1, 2)
Has data? -> True


In [8]:
# Now we are going to print all the configuration of the TT-Ultra at the moment of the measurement
config = fr.getConfiguration()

for key, value in config.items():
    print(f"{key}: {value}")

channel number scheme: 2
configure version: 1
current time: 2025-02-20 15:53:51 +0100
devices: [{'FPGA ID': 11493476189726804, 'PCB UID': '00fdf287', 'PCB version': '1.8 (1)', 'minimum software version': '2.10.6', 'model': 'Time Tagger Ultra', 'serial': '2410001ABQ'}]
firmware version: TT-Ultra, FW6, TS 2024-02-09 12:21:10, OK 1.46
fpga link: {'enabled': False}
hardware buffer size: 67108864
inputs: [{'average rising falling': [False, False], 'channel': [1, -1], 'conditional filter filtered': [False, False], 'conditional filter triggers': [False, False], 'deadtime': [2000, 2000], 'delay hardware': [0, 0], 'delay software': [0, 0], 'event divider': [1, 1], 'hardware delay compensation': [333, 333], 'high priority channel': [False, False], 'input hysteresis': 0, 'input impedance high': False, 'input mux': 0, 'normalization': [True, True], 'resolution': 'Standard', 'resolution rms': 42.0, 'trigger level': 0.5}, {'average rising falling': [False, False], 'channel': [2, -2], 'conditional fi

### Some useful code for obtaining the timestamps on each channel

In [9]:
data = fr.getData(10)
# Reads the next n_events and returns the buffer object with the specified number of timetags.
print(fr.hasData())
# Returns: True if more data is available for reading, False if all data has been read from all the files specified in the class constructor.

print(data.getTimestamps())
# Returns: Event timestamps in picoseconds for all chosen channels.
print(data.getChannels())
# Returns: Channel number for each detected event.

print(data.tGetData)
# Returns: The data-stream time position when the TimeTagStream or FileWriter started data acquisition.
print(data.tStart)


data = fr.getData(4)
print(fr.hasData())
print(data.getTimestamps())
print(data.getChannels())

print(fr.getData(1).getTimestamps()-fr.getData(1).getTimestamps())

True
[1041964272719608 1041964605905306 1041964611249229 1041964629901671
 1041964676300824 1041964846712562 1041964872313087 1041964881906598
 1041964901249119 1041964924721652]
[1 2 2 1 1 1 1 1 2 2]
1041964930292740
1041964220416000
True
[1041964930292740 1041964988274899 1041965228204165 1041965241120760]
[1 2 1 2]
[-103943577]


In [53]:
virtual_tagger = createTimeTaggerVirtual()
virtual_tagger.getConfiguration()

# measure photon antibunching
corr_ch1 = 1 # first photon channel for antibunching measurements
corr_ch2 = 2 # second photon channel for antibunching measurements
bwcorr = 1000 # 1 ns
nbins = 1000

cr = Countrate(virtual_tagger, [corr_ch1, corr_ch2])

virtual_tagger.setReplaySpeed(-1.0) # Speed of -1.0 will replay the data as fast as possible
virtual_tagger.replay(file, begin = 0, duration = -1) # collect data until the end 
virtual_tagger.waitForCompletion()

print(f"Rate of counts channel {corr_ch1} (counts/sec) : {cr.getData()[0]}")
print(f"Rate of counts channel {corr_ch2} (counts/sec) : {cr.getData()[1]}")

Rate of counts channel 1 (counts/sec) : 8003.999108212611
Rate of counts channel 2 (counts/sec) : 4305.408941877971
